#Googleドライブにマウント
ランタイムがGPUに設定されているか確認すること　\
パスやディレクトリなどは自分の環境にあわせて変えていく必要がある\
※初回は4行目をコメントアウトし、5-7行目のコメントアウトを外す\
　また、2回目以降は5-7行目をコメントアウトし、4行目のコメントアウトを外す

In [1]:
from google.colab import drive
drive.mount('/content/drive')
path = '/content/drive/My Drive/ColabData'
%cd  /content/drive/My Drive/ColabData/Mask_RCNN
#%cd  /content/drive/My Drive/ColabData
#!git clone https://github.com/matterport/Mask_RCNN.git
#cd Mask_RCNN

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/ColabData/Mask_RCNN


#必要なパッケージをインストール
最初にやってしまえば2回目以降は不要




In [2]:
#!pip install -r requirements.txt

In [3]:
#!git clone https://github.com/waleedka/coco.git
#%cd coco/PythonAPI/
#!python setup.py build_ext install

In [4]:
#%cd ../..
#!python setup.py install

#tensorflow及びkerasのバージョン指定
tensorは1．x系でないとmaskrcnnは動かないので設定
またkerasのバージョンも指定　\
※もしうまくいかない時はランタイムを再起動する

In [2]:
%tensorflow_version 1.x
!pip install keras==2.2.5

TensorFlow 1.x selected.


#Configクラスの作成
エポックあたりのステップ数などは好きな値に変更していくこと

In [3]:
from mrcnn.config import Config


class OneClassConfig(Config):

    #: config名
    NAME = "leaf_dataset"

    #: バッチあたり画像数 (GPUのメモリが大きいなら増やしてもよい
    IMAGES_PER_GPU = 1

    # クラス数　= 背景 + 検出クラス数
    NUM_CLASSES = 1 + 1

    # エポックあたりステップ数
    STEPS_PER_EPOCH = 50

    VALIDATION_STEPS = 5

    # 提案領域のconfidenceが90%以下なら物体検出フェイズをスキップ
    DETECTION_MIN_CONFIDENCE = 0.9

#Datasetクラスの作成
データセットは、Mask-RCNNフォルダの中に以下のようなフォルダを作る　


---


Mask-RCNN \
　|- dataset \
　　　|- train \
　　　|　　|- image \
　　　|　　|　|- 1.jpg\
　　　|　　|　|- 2.jpg\
　　　|　　|　L ... \
　　　|　　L mask \
　　　|　　　 |- 1.jpg \
　　　|　　　 |- 2.jpg \
　　　|　　　 L ... \
　　　L valid \
　　　　　 |- image \
　　　　　 |　|- 1.jpg\
　　　　　 |　|- 2.jpg\
　　　　　 |　L ... \
　　　　　 L mask \
　　　　　　  |- 1.jpg\
　　　　　　  |- 2.jpg\
　　　　　　  L ... 　　


---


注意事項

1.   imageフォルダとmaskフォルダ内の画像の名前は一致するようにすること
2.   jpg画像を利用すること(ほかの形式には対応していない)



In [4]:
import pathlib

import cv2
import numpy as np
from PIL import Image
from mrcnn import utils
from mrcnn.model import log
import copy


class OneClassDataset(utils.Dataset):

    def load_dataset(self, dataset_dir):
        """ データセットを登録
        """
        #: データセット名、クラスID、クラス名
        self.add_class('leaf_dataset', 1, 'leaf')
        
        #images = glob.glob("/content/drive/My Drive/ColabData/Mask_RCNN/dataset/train/image/*.jpg")
        #if dataset_dir == '/content/drive/My Drive/ColabData/Mask_RCNN/dataset/train':
        #  for i in range(1,4):
        #    images = glob.glob(os.path.join(dataset_dir, "image",str(i)+".jpg"))
        #print(dataset_dir)
        images = glob.glob(os.path.join(dataset_dir, "image","*.jpg"))
        print(images)
        #masks = glob.glob("/content/drive/My Drive/ColabData/Mask_RCNN/dataset/train/mask/*.jpg")
        masks = glob.glob(os.path.join(dataset_dir, "mask","*.jpg"))
        print(masks)

        for image_path, mask_path in zip(images, masks):
            image_path = pathlib.Path(image_path)
            mask_path = pathlib.Path(mask_path)
            #print(mask_path)
            assert image_path.name == mask_path.name, 'データセット名不一致'

            image = Image.open(image_path)
            height = image.size[0]
            width = image.size[1]

            mask = Image.open(mask_path)
            assert image.size == mask.size, "サイズ不一致"

            self.add_image(
                'leaf_dataset',
                path=image_path,
                image_id=image_path.stem,
                mask_path=mask_path,
                width=width, height=height)


    def blob_detection(self, mask_path):
        mask = cv2.imread(mask_path, 0)
        #: 念のためもう一度二値化
        _, mask = cv2.threshold(mask, 150, 255, cv2.THRESH_BINARY)

        label = cv2.connectedComponentsWithStats(mask)
        data = copy.deepcopy(label[1])

        labels = []
        for label in np.unique(data):
            #: ラベル0は背景
            if label == 0:
                continue
            else:
                labels.append(label)

        mask = np.zeros((mask.shape)+(len(labels),), dtype=np.uint8)

        for n, label in enumerate(labels):
            mask[:, :, n] = np.uint8(data == label)

        cls_idxs = np.ones([mask.shape[-1]], dtype=np.int32)

        return mask, cls_idxs

    def load_mask(self, image_id):
        """マスクデータとクラスidを生成する
        """
        image_info = self.image_info[image_id]
        if image_info["source"] != 'leaf_dataset':
            return super(self.__class__, self).load_mask(image_id)

        mask_path = image_info['mask_path']
        mask, cls_idxs = self.blob_detection(str(mask_path))

        return mask, cls_idxs

    def image_reference(self, image_id):
        """Return the path of the image."""
        info = self.image_info[image_id]
        if info["source"] == 'leaf_dataset':
            return info
        else:
            super(self.__class__, self).image_reference(image_id)



Using TensorFlow backend.


#学習の実行
データセットへのパスはそれぞれ自分に合わせて設定すること

In [ ]:
import os
import glob
import mrcnn.model as modellib
from mrcnn import utils


TRAIN_DATASET = os.path.join("/content/drive/My Drive/ColabData/Mask_RCNN/dataset/train")
#TRAIN_DATASET = os.path.join('dataset', 'train')
dataset_train = OneClassDataset()
dataset_train.load_dataset(TRAIN_DATASET)
dataset_train.prepare()

VALID_DATASET = os.path.join("/content/drive/My Drive/ColabData/Mask_RCNN/dataset/valid")
#VALID_DATASET = os.path.join('datset', 'valid')
dataset_val = OneClassDataset()
dataset_val.load_dataset(VALID_DATASET)
dataset_val.prepare()

config = OneClassConfig()
#config.display()

model = modellib.MaskRCNN(mode="training", model_dir="logs/model",
                          config=config)

COCO_MODEL_PATH = 'mask_rcnn_coco.h5'
if not os.path.exists(COCO_MODEL_PATH):
    utils.download_trained_weights(COCO_MODEL_PATH)

model.load_weights(COCO_MODEL_PATH, by_name=True,
                    exclude=["mrcnn_class_logits", "mrcnn_bbox_fc",
                            "mrcnn_bbox", "mrcnn_mask"])

In [ ]:
#: ネットワークのhead部分のみの訓練
model.train(dataset_train, dataset_val,
            learning_rate=0.001,
            epochs=30,
            layers='heads')

#validデータで推論を行い結果を表示(保存)

In [7]:
class InferenceConfig(OneClassConfig):
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1
    DETECTION_MIN_CONFIDENCE = 0.5

In [ ]:
    import random
    import skimage.io
    from mrcnn.visualize import display_images
    from mrcnn import visualize
    from mrcnn.model import log
    from PIL import Image
    import matplotlib.pyplot as plt

    VALID_DATASET = os.path.join('dataset', 'valid')
    dataset_val = OneClassDataset()
    dataset_val.load_dataset(VALID_DATASET)
    dataset_val.prepare()

    config = InferenceConfig()
    MODEL_DIR = os.path.join("logs", "model")
    model = modellib.MaskRCNN(mode="inference", model_dir=MODEL_DIR,
                              config=config)
    
    weights_path = model.find_last()
    print("Loading weights ", weights_path)
    model.load_weights(weights_path, by_name=True)

    
    #id = random.choice(dataset_val.image_ids)　#ランダムな1枚の画像を処理するとき
    for image_id in dataset_val.image_ids:
      image, image_meta, gt_class_id, gt_bbox, gt_mask = modellib.load_image_gt(dataset_val, config, image_id)
      info = dataset_val.image_info[image_id]
    
      results = model.detect([image], verbose=1)
      #ax = get_ax(1)
      r = results[0]

      """
      # 結果を表示する場合はこちらを使う
      visualize.display_instances(image, r['rois'], r['masks'], r['class_ids'],
                                                 dataset_val.class_names, r['scores'], show_bbox=True, show_mask=True,
                                                 title="Predictions")
      """
      """  
      #結果を保存する場合はこちらを使う
      N = r['rois'].shape[0]
      result_image = image
      colors = visualize.random_colors(N)
      for i in range(N):
        #Color
        color = colors[i]
        rgb = (round(color[0] * 255), round(color[1] * 255), round(color[2] * 255))
        
        #Rect
        result_image = visualize.draw_box(result_image, r['rois'][i], rgb)
        
        #Label & Score
        font = cv2.FONT_HERSHEY_SIMPLEX
        text = dataset_val.class_names[r['class_ids'][i]] + ':' + str(r['scores'][i])
        result_image = cv2.putText(result_image, text,
            (r['rois'][i][1],r['rois'][i][0]), font, 0.8, rgb, 2, cv2.LINE_AA)
        #print(class_names[r['class_ids'][i]])
        
        #Mask
        mask = r['masks'][:, :, i]
        result_image = visualize.apply_mask(result_image, mask, color)

        result_image = cv2.cvtColor(result_image, cv2.COLOR_BGR2RGB)
        #result_image.save("/content/drive/My Drive/ColabData/Mask_RCNN/dataset/result/" + str(image_id) + ".jpg")
        #skimage.io.imshow(result_image)
        #plt.show()                                        
        cv2.imwrite("/content/drive/My Drive/ColabData/Mask_RCNN/dataset/result/" + str(image_id) + ".jpg",result_image)
      """
      """
      r['rois']には矩形の座標
      r['masks']にはrois内でmaskをかける座標
      r['class_ids']には検知した物体の名前
      r['scores']には物体の検知精度 
      """
